In [ ]:
from confluent_kafka.avro import AvroConsumer
from confluent_kafka.avro.serializer import SerializerError
import json

# Kafka configuration
bootstrap_servers = '10.24.36.25:35002'
schema_registry_url = 'http://10.24.36.25:35003'
topic_name = 'table-update'

# Create an AvroConsumer
consumer = AvroConsumer({
    'bootstrap.servers': bootstrap_servers,
    'group.id': 'table-update',
    'schema.registry.url': schema_registry_url,
    'auto.offset.reset': 'earliest'
})

consumer.subscribe([topic_name])

# File to store JSON data
output_file = 'kafka_messages.json'
messages = []

try:
    message_count = 0
    while message_count < 3:
        try:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue

            print(f"Received message: {msg.value()}")
            messages.append(msg.value())
            message_count += 1

        except SerializerError as e:
            print(f"Message deserialization failed: {e}")
            break
finally:
    consumer.close()

# Write messages to a JSON file
with open(output_file, 'w') as file:
    json.dump(messages, file, indent=4)

print(f"Consumed {message_count} messages and written to {output_file}")
